In [8]:
from analysis import MatchAnalysis
import util_strings as utils
from ranking import Ranking


In [9]:
ranking = Ranking('SerieA', '2022-2023')
ranking.read_matches(seasons=4, path=utils.ranking)

ma = MatchAnalysis()
ma.set_ranking(ranking=ranking)

In [10]:
#leggo tutti i match (doppi perché ci sono le statistiche x ogni squadra)
ma.read_matches(utils.merged_statistics) 
#creo una lista di oggetti (lista di squadre)
ma.create_team_dataset()
#divido le partite di casa da quelle in trasferta
ma.divide_and_merge_home_away()
#vado a combinarle
ma.reduce_dataset_with_avg(number = 5, path=utils.dataset_without_text)

In [14]:
import pandas as pd
ma.diff_dataset['date'] = pd.to_datetime(ma.diff_dataset['date'], format='%Y-%m-%d') 

for i, match_value in ma.diff_dataset.iterrows():
    season = match_value.season.split('-')[1]
    team1, team2 = (0, 0)
    seasons_to_check = [x for x in sorted(ranking.all_previous_seasons_matches.keys()) if x <= int(season)]
    
    #per il rank ho bisogno dei match nelle stagioni precedenti, quindi scansiono ogni stagione e sommo i rank
    for season_to_check in seasons_to_check: 
        matches = ranking.all_previous_seasons_matches[season_to_check] 
        teams = matches.Casa.unique()
        if match_value.home not in teams and match_value.away not in teams:
            team1, team2 = team1, team2 
        elif match_value.home not in teams and match_value.away in teams:
            team1, team2 = team1, team2 + 2
        elif match_value.home in teams and match_value.away not in teams:
            team1, team2 = team1 + 2, team2 
        else:
            v1, v2 = ranking.get_result(matches, match_value.home, match_value.away, match_value.date)
    
            team1 += v1
            team2 += v2
            
    ma.diff_dataset.at[i, 'rank_h'] = team1
    ma.diff_dataset.at[i, 'rank_a'] = team2

In [13]:
ma.diff_dataset[((ma.diff_dataset.home=='Inter')&(ma.diff_dataset.away=='Juventus'))|((ma.diff_dataset.home=='Juventus')&(ma.diff_dataset.away=='Inter'))]

,home,date,result,goals,away,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards,season,rank_h,rank_a
140,Juventus,2018-12-07,V,-0.4,Inter,-3.0,-2.2,0.2,0.2,-5.8,3.0,-0.4,-0.2,2018-2019,0.0,0.0
330,Inter,2019-04-27,N,0.0,Juventus,2.2,1.0,0.2,0.2,14.4,-4.6,-0.4,0.0,2018-2019,0.0,1.0
447,Inter,2019-10-06,P,-0.2,Juventus,-0.2,0.4,0.0,0.0,-0.6,-2.2,0.2,0.2,2019-2020,0.0,1.0
630,Juventus,2020-03-08,V,0.0,Inter,0.2,-1.0,0.2,0.2,8.6,2.8,0.0,-0.2,2019-2020,2.0,0.0
935,Inter,2021-01-17,V,-0.2,Juventus,-1.0,-0.8,0.2,0.4,-5.6,0.0,-0.2,-0.2,2020-2021,0.0,3.0
1120,Juventus,2021-05-15,V,-0.6,Inter,-2.4,-1.2,0.0,0.0,-0.4,8.4,1.0,0.0,2020-2021,3.0,1.0
1227,Inter,2021-10-24,N,1.0,Juventus,2.6,0.6,0.2,0.4,-3.2,-1.6,0.8,0.0,2021-2022,1.0,4.0
1440,Juventus,2022-04-03,P,0.6,Inter,-9.8,-2.8,0.2,0.2,-7.8,-0.4,-0.2,0.0,2021-2022,4.0,1.0
